In [1]:
import PyPDF2

pdfFileObj = open('Quarterly Press Release.pdf','rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
print(pdfReader.pages)
pageObj = pdfReader.pages[0]
print(pageObj.extract_text())

 
Scotiabank  Second  Quarter  Press  Release  2024    1  
 
Second  Quarter  2024  Earnings  Release  
 
Scotiabank  reports  second  quarter  results  
 
All amounts  are  in Canadian  dollars  and  are  based  on  our  unaudited  Interim  Condensed  Co nsolidated  Financial  Statements  for  the  quarter  ended  April  30,  2024  and  related  
notes  prepared  in accordance  with  International  Financial  Reporting  Standards  (IFRS)  as issued  by the  International  Accounting  Standards  Board  (IASB),  unless  otherwise  
noted.  Our  complete  Second  Quarter  2024  Report  to Shareholders,  including  our  unaudited  interim  financial  statements  for  the  period  ended  April  30,  2024,  can  also  be  
found  on  the  SEDAR+  website  at www.sedarplus.ca  and  on  the  EDGAR  section  of the  SEC’s  website  at www.sec.gov . Supplementary  Financial  Information  is also  available,  
together  with  the  Second  Quarter  2024  Report  to Shareholders  on  the  Investor

In [2]:
pdfFileObj = open('Quarterly Press Release.pdf','rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
print(pdfReader.pages)
pageObj = pdfReader.pages[1]
print(pageObj.extract_text())

 
2   Scotiabank  Second  Quarter  Press  Release  2024  Financial  Highlights  
Reported  Results  For  the  three  months  ended  For  the  six months  ended  
 April  30   January  31  April  30   April  30   April  30  
(Unaudited)  ($ millions)   2024(1)   2024(1)   2023(1)   2024(1)   2023(1) 
Operating  results                
Net  interest  income  $ 4,694   $ 4,773   $ 4,460   $ 9,467   $ 9,023  
Non -interest  income   3,653    3,660    3,453    7,313    6,852  
Total  revenue  $ 8,347   $ 8,433   $ 7,913   $ 16,780   $ 15,875  
Provision  for  credit  losses   1,007    962    709    1,969    1,347  
Non -interest  expenses   4,711    4,739    4,574    9,450    9,035  
Income  tax  expense   537    533    484    1,070    1,589  
Net  income  $ 2,092   $ 2,199   $ 2,146   $ 4,291   $ 3,904  
Net  income  attributable  to non -controlling  interests  in subsidiaries   26    25   24   51   61 
Net  income  attributable  to equity  holders  of the  Bank  $ 2,066   $ 2,174   $ 2,1

In [3]:
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import io

In [4]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
        text = fake_file_handle.getvalue()
    # close open handles
    converter.close()
    fake_file_handle.close()
    if text:
      return text

In [5]:
from pdfminer.high_level import extract_text
def extract_text_from_pdf(pdf_path):
    laparams = LAParams()
    text = extract_text(pdf_path,laparams)
    return text.split("\f")

In [6]:
extract_text_from_pdf('Quarterly Press Release.pdf')

['Second Quarter 2024 Earnings Release \n\nScotiabank reports second quarter results \n\nAll amounts are in Canadian dollars and are based on our unaudited Interim Condensed Consolidated Financial Statements for the quarter ended April 30, 2024 and related \nnotes prepared in accordance with International Financial Reporting Standards (IFRS) as issued by the International Accounting Standards Board (IASB), unless otherwise \nnoted. Our complete Second Quarter 2024 Report to Shareholders, including our unaudited interim financial statements for the period ended April 30, 2024, can also be \nfound on the SEDAR+ website at www.sedarplus.ca and on the EDGAR section of the SEC’s website at www.sec.gov. Supplementary Financial Information is also available, \ntogether with the Second Quarter 2024 Report to Shareholders on the Investor Relations page at www.scotiabank.com. \n\nSecond Quarter 2024 Highlights on a Reported Basis  \n(versus Q2 2023) \n•  Net income of $2,092 million, compared to

In [7]:
import re
def extract_text_between_words_across_pages(text_pages, start_word, end_word):
    extracted_texts = []
    combined_text = "\n".join(text_pages)
    pattern = re.compile(re.escape(start_word) + '(.*?)' + re.escape(end_word), re.S)

    for match in pattern.finditer(combined_text):
        start_page, end_page = None, None
        start_index, end_index = match.start(), match.end()

        cumulative_length = 0
        for i, page in enumerate(text_pages):
            page_length = len(page)
            if start_page is None and start_index < cumulative_length + page_length:
                start_page = i + 1
            if end_page is None and end_index <= cumulative_length + page_length:
                end_page = i + 1
            cumulative_length += page_length + len("\n")
            if start_page is not None and end_page is not None:
                break

        extracted_texts.append({
            'start_page': start_page,
            'end_page': end_page,
            'start_word': start_word,
            'end_word': end_word,
            'extracted_text': match.group(1).strip()
        })
    return extracted_texts

In [8]:
start_end_pairs = [
    ('Canadian Banking', 'non-interest expenses'),
    # Add more pairs as needed
]

import os
def process_files_in_folder(file_name, start_end_pairs):
    all_extracted_texts = []
    if file_name.endswith('.pdf'):
      text_pages = extract_text_from_pdf(file_name)
      for start_word, end_word in start_end_pairs:
        extracted_texts = extract_text_between_words_across_pages(text_pages, start_word, end_word)
        all_extracted_texts.extend(extracted_texts)

    return all_extracted_texts

all_extracted_texts = process_files_in_folder('Quarterly Press Release.pdf', start_end_pairs)

In [9]:
all_extracted_texts

[{'start_page': 1,
  'end_page': 2,
  'start_word': 'Canadian Banking',
  'end_word': 'non-interest expenses',
  'extracted_text': 'delivered adjusted earnings(1) of $1 billion this quarter. Solid revenue growth outpaced expense growth resulting in another quarter of \npositive operating leverage, while provision for credit losses increased compared to the prior year. In addition, deposit growth, a key component of the \nrefreshed strategy, was up 7% year-over-year. \n\nInternational Banking generated adjusted earnings(1) of $701 million. Revenue growth driven by strong margin expansion, disciplined expense and capital \nmanagement, were offset by higher provision for credit losses. Adjusted return on equity(1) was 14.5%, a 120 basis point improvement from last year. \n\nGlobal Wealth Management adjusted earnings(1) were $389 million, up 8% year over year. Assets under management(2) of $349 billion increased by 6% \nresulting in strong revenue growth, partly offset by investments to su

In [10]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import re

def read_article(article):
    sentences = []
    for sentence in article.split('. '):
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split())
    return sentences

def sentence_similarity(sent1, sent2, stop_words=None):
    if stop_words is None:
        stop_words = []

    sent1 = [w.lower() for w in sent1 if w not in stop_words]
    sent2 = [w.lower() for w in sent2 if w not in stop_words]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # Build the vector for the first sentence
    for w in sent1:
        if w in stop_words:
            continue
        vector1[all_words.index(w)] += 1

    # Build the vector for the second sentence
    for w in sent2:
        if w in stop_words:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:  # ignore if both are the same sentence
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

def generate_summary(article, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text and split it
    sentences = read_article(article)

    # Step 2 - Generate Similarity Matrix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    print("Indexes of top ranked_sentence order are ", ranked_sentence)

    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Output the summarize text
    print("Summarized Text: \n", ". ".join(summarize_text))


In [11]:
generate_summary(all_extracted_texts[0] ['extracted_text'], 2)

Indexes of top ranked_sentence order are  [(0.06521064231231842, ['Solid', 'revenue', 'growth', 'outpaced', 'expense', 'growth', 'resulting', 'in', 'another', 'quarter', 'of', 'positive', 'operating', 'leverage,', 'while', 'provision', 'for', 'credit', 'losses', 'increased', 'compared', 'to', 'the', 'prior', 'year']), (0.057835613181881886, ['Revenue', 'growth', 'driven', 'by', 'strong', 'margin', 'expansion,', 'disciplined', 'expense', 'and', 'capital', 'management,', 'were', 'offset', 'by', 'higher', 'provision', 'for', 'credit', 'losses']), (0.05507177859767471, ['Refer', 'to', 'Note', '4', 'of', 'the', 'condensed', 'interim', 'consolidated', 'financial', 'statements', 'in', 'the', 'Bank’s', 'Q2', '2024', 'Quarterly', 'Report', 'to', 'Shareholders']), (0.05426351789624204, ['Refer', 'to', 'Notes', '3', 'and', '4', 'of', 'the', 'condensed', 'interim', 'financial', 'statements', 'in', 'the', 'Bank’s', 'Q2', '2024', 'Quarterly', 'Report', 'to', 'Shareholders', 'for', 'details']), (0.05

In [12]:
print(len(pdfReader.pages))

11


In [13]:
text = pageObj.extract_text()
text

' \n2   Scotiabank  Second  Quarter  Press  Release  2024  Financial  Highlights  \nReported  Results  For  the  three  months  ended  For  the  six months  ended  \n April  30   January  31  April  30   April  30   April  30  \n(Unaudited)  ($ millions)   2024(1)   2024(1)   2023(1)   2024(1)   2023(1) \nOperating  results                \nNet  interest  income  $ 4,694   $ 4,773   $ 4,460   $ 9,467   $ 9,023  \nNon -interest  income   3,653    3,660    3,453    7,313    6,852  \nTotal  revenue  $ 8,347   $ 8,433   $ 7,913   $ 16,780   $ 15,875  \nProvision  for  credit  losses   1,007    962    709    1,969    1,347  \nNon -interest  expenses   4,711    4,739    4,574    9,450    9,035  \nIncome  tax  expense   537    533    484    1,070    1,589  \nNet  income  $ 2,092   $ 2,199   $ 2,146   $ 4,291   $ 3,904  \nNet  income  attributable  to non -controlling  interests  in subsidiaries   26    25   24   51   61 \nNet  income  attributable  to equity  holders  of the  Bank  $ 2,066   

In [14]:
text = text.replace('\n', ' ')
text

'  2   Scotiabank  Second  Quarter  Press  Release  2024  Financial  Highlights   Reported  Results  For  the  three  months  ended  For  the  six months  ended    April  30   January  31  April  30   April  30   April  30   (Unaudited)  ($ millions)   2024(1)   2024(1)   2023(1)   2024(1)   2023(1)  Operating  results                 Net  interest  income  $ 4,694   $ 4,773   $ 4,460   $ 9,467   $ 9,023   Non -interest  income   3,653    3,660    3,453    7,313    6,852   Total  revenue  $ 8,347   $ 8,433   $ 7,913   $ 16,780   $ 15,875   Provision  for  credit  losses   1,007    962    709    1,969    1,347   Non -interest  expenses   4,711    4,739    4,574    9,450    9,035   Income  tax  expense   537    533    484    1,070    1,589   Net  income  $ 2,092   $ 2,199   $ 2,146   $ 4,291   $ 3,904   Net  income  attributable  to non -controlling  interests  in subsidiaries   26    25   24   51   61  Net  income  attributable  to equity  holders  of the  Bank  $ 2,066   $ 2,174   $ 2,

In [15]:
c = 0
text = ""
while c < len(pdfReader.pages):
    pageObj = pdfReader.pages[c]
    c += 1
    text += pageObj.extract_text()
print(text)

 
Scotiabank  Second  Quarter  Press  Release  2024    1  
 
Second  Quarter  2024  Earnings  Release  
 
Scotiabank  reports  second  quarter  results  
 
All amounts  are  in Canadian  dollars  and  are  based  on  our  unaudited  Interim  Condensed  Co nsolidated  Financial  Statements  for  the  quarter  ended  April  30,  2024  and  related  
notes  prepared  in accordance  with  International  Financial  Reporting  Standards  (IFRS)  as issued  by the  International  Accounting  Standards  Board  (IASB),  unless  otherwise  
noted.  Our  complete  Second  Quarter  2024  Report  to Shareholders,  including  our  unaudited  interim  financial  statements  for  the  period  ended  April  30,  2024,  can  also  be  
found  on  the  SEDAR+  website  at www.sedarplus.ca  and  on  the  EDGAR  section  of the  SEC’s  website  at www.sec.gov . Supplementary  Financial  Information  is also  available,  
together  with  the  Second  Quarter  2024  Report  to Shareholders  on  the  Investor

In [16]:
special_chars = ['•', '●', '▪', '•', '◦', '-', '–', '—']
for char in special_chars:
    text = text.replace(char, '')
text

" \nScotiabank  Second  Quarter  Press  Release  2024    1  \n \nSecond  Quarter  2024  Earnings  Release  \n \nScotiabank  reports  second  quarter  results  \n \nAll amounts  are  in Canadian  dollars  and  are  based  on  our  unaudited  Interim  Condensed  Co nsolidated  Financial  Statements  for  the  quarter  ended  April  30,  2024  and  related  \nnotes  prepared  in accordance  with  International  Financial  Reporting  Standards  (IFRS)  as issued  by the  International  Accounting  Standards  Board  (IASB),  unless  otherwise  \nnoted.  Our  complete  Second  Quarter  2024  Report  to Shareholders,  including  our  unaudited  interim  financial  statements  for  the  period  ended  April  30,  2024,  can  also  be  \nfound  on  the  SEDAR+  website  at www.sedarplus.ca  and  on  the  EDGAR  section  of the  SEC’s  website  at www.sec.gov . Supplementary  Financial  Information  is also  available,  \ntogether  with  the  Second  Quarter  2024  Report  to Shareholders  on  t

In [17]:
text = ' '.join(text.split())
text

"Scotiabank Second Quarter Press Release 2024 1 Second Quarter 2024 Earnings Release Scotiabank reports second quarter results All amounts are in Canadian dollars and are based on our unaudited Interim Condensed Co nsolidated Financial Statements for the quarter ended April 30, 2024 and related notes prepared in accordance with International Financial Reporting Standards (IFRS) as issued by the International Accounting Standards Board (IASB), unless otherwise noted. Our complete Second Quarter 2024 Report to Shareholders, including our unaudited interim financial statements for the period ended April 30, 2024, can also be found on the SEDAR+ website at www.sedarplus.ca and on the EDGAR section of the SEC’s website at www.sec.gov . Supplementary Financial Information is also available, together with the Second Quarter 2024 Report to Shareholders on the Investor Relations page at www.scotiabank.com . Second Quarter 2024 Highlights on a Reported Basis (versus Q2 2023) Second Quarter 2024 